# Install

In [ ]:
import os

# check if `pytorch-CycleGAN-and-pix2pix` is already cloned
if not os.path.exists('pix2pixHD'):
	!git clone https://github.com/NVIDIA/pix2pixHD.git
else:
    print('pix2pixHD already cloned')

In [ ]:
os.chdir('pix2pixHD')

In [ ]:
!pwd

In [ ]:
%pip install -q dominate

# Checkpoints

In [ ]:
if not os.path.exists('checkpoints/fish_pix2pix'):
    !bash ../scripts/download_pix2pixHD_pretrained.sh
else:
    print('Pretrained model already exists')

# Check Testing Dataset

In [ ]:
import os

try:
    os.chdir('../dataset')
    if not os.path.exists('test_pose'):
        !bash ../scripts/download_test_pose.sh
    else:
        print('Test pose already exists')
except:
    print('Error: dataset folder not found')


In [ ]:
# move the folder to model folder
import shutil

try:
    os.chdir('..')
except:
    print('Error: Already in the root folder')


source_folder = './dataset/test_pose'
target_folder = './pix2pixHD/datasets/test_pose'

if not os.path.exists(target_folder):
    shutil.move(source_folder, target_folder)
else:
    print('Test pose already exists in pix2pixHD folder')

## Rename the test_pose subfolder

In [ ]:
os.rename('./pix2pixHD/datasets/test_pose/testA', './pix2pixHD/datasets/test_pose/test_A')

In [ ]:
# change os path
os.chdir('pix2pixHD')

In [ ]:
!pwd

# Testing

In [ ]:
!python test.py --dataroot datasets/test_pose --name fish_pix2pix --label_nc 0 --no_instance --netG global --resize_or_crop none --how_many 100

# Combine all the generated images into a video

In [ ]:
import cv2
from PIL import Image
import numpy as np

images_path = 'results/fish_pix2pix/test_latest/images/'
output_path = 'results/fish_pix2pix/videos/'
output_width, output_height = 1280, 720

# check if the output path exists, if not create it
if not os.path.exists(output_path):
    os.makedirs(output_path)

image_array = os.listdir(images_path)
# only keep the images that with 'fake_B' in the name
image_array = [x for x in image_array if '_synthesized_image' in x]
image_array.sort(key=lambda x: int(x.split('_')[0]))
# print(image_array)

img_array = []
for filename in image_array:
    img = Image.open(f'{images_path}/{filename}')
    # resize the image
    img = img.resize((output_width, output_height), resample=Image.BICUBIC)
    img.save(f'{output_path}/{filename}')
    img = np.array(img)
    img_array.append(img)

print(f"Number of frames: {len(img_array)}")
print(f"Frame size: {(output_width, output_height)}")
# print(img_array)

In [ ]:
from moviepy.editor import VideoFileClip, AudioFileClip, ImageSequenceClip

clip = ImageSequenceClip(img_array, fps=30)  # Adjust fps as needed

# set audio
audio = AudioFileClip('../src/magnetic.mp4')

clip = clip.set_audio(audio)

# check if the output path exists, if not create it
if not os.path.exists('../src/pix2pixHD'):
    os.makedirs('../src/pix2pixHD')

clip.write_videofile(f'../src/pix2pixHD/magnetic_fish.mp4', codec='libx264', audio_codec="aac", fps=30)

# Combine the video

combine 3 imgs into a video
- real_world_dance
- real_world_pose
- generated_video

In [ ]:
dance_folder = 'datasets/test_pose/source'
pose_folder = 'datasets/test_pose/test_A'
gen_img_array = img_array

# sort the dance_folder
dance_img_array = os.listdir(dance_folder)
dance_img_array.sort(key=lambda x: int(x.split('.')[0]))

# sort the pose_folder
pose_img_array = os.listdir(pose_folder)
pose_img_array.sort(key=lambda x: int(x.split('.')[0]))

print(f"Number of totall images: {len(dance_img_array)}")

# combine the test and generated images
combined_img_array = []
for dance_img, pose_img, gen_img in zip(dance_img_array, pose_img_array, gen_img_array):
    dance_img = Image.open(f'{dance_folder}/{dance_img}')
    pose_img = Image.open(f'{pose_folder}/{pose_img}')
    gen_img = Image.fromarray(gen_img)
    combined_img = Image.new('RGB', (3*output_width, output_height))
    combined_img.paste(dance_img, (0, 0))
    combined_img.paste(pose_img, (output_width, 0))
    combined_img.paste(gen_img, (2*output_width, 0))
    combined_img = np.array(combined_img)
    combined_img_array.append(combined_img)
    
print(f"Number of combined images: {len(combined_img_array)}")

# Write the video out

In [ ]:
from moviepy.editor import ImageSequenceClip

clip = ImageSequenceClip(combined_img_array, fps=30)  # Adjust fps as needed

audio_clip = AudioFileClip("../src/magnetic.mp4")

clip = clip.set_audio(audio_clip)

clip.write_videofile(f'../src/pix2pixHD/magnetic_dance2pose2fish.mp4', codec='libx264', audio_codec="aac",  fps=30)